# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
%matplotlib inline
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
C:\Users\sumit\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [4]:
# Create our features
X =df.copy()
X.drop("loan_status",axis=1,inplace=True)

# Create our target
y = df['loan_status']

In [5]:
# X.info()

In [6]:
from sklearn.preprocessing import LabelEncoder
# to transform the homeowner column from object to int64 i.e. into 0,1,2
le = LabelEncoder()
le.fit(X['homeowner'])
X['homeowner']=le.transform(X['homeowner'])



In [7]:
X.homeowner.unique()

array([1, 2, 0], dtype=int64)

In [8]:
y[:5]

0    low_risk
1    low_risk
2    low_risk
3    low_risk
4    low_risk
Name: loan_status, dtype: object

In [9]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [10]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)

In [11]:
# X_train.info(verbose=True)

In [12]:
# X_test.describe()


In [13]:
y_train.describe()

count        58152
unique           2
top       low_risk
freq         56244
Name: loan_status, dtype: object

In [14]:
y_test.shape

(19384,)

In [15]:
y_train.shape

(58152,)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [16]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()


In [17]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler=scaler.fit(X_train)

In [18]:
# Scale the training and testing data
X_train_scaled=X_scaler.transform(X_train)
X_test_scaled=X_scaler.transform(X_test)


# Simple Logistic Regression

In [19]:
# used X_train_scaled in the logistic regerssion as the data has been scaled 

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [20]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9879955386420904

In [21]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  581,    11],
       [  102, 18690]], dtype=int64)

In [22]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      0.98      0.99      0.91      0.99      0.97       592
   low_risk       1.00      0.99      0.98      1.00      0.99      0.98     18792

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



<p>Comment * to check if this is the correct thinking *<p> <li> The recall of 0.99 means all the cases which were required to be classified as low-risk were correctly classified. <li>Further, precision of 0.85 means high-risk were classified upto 85% correctly.

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [23]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros=RandomOverSampler(random_state=1)
X_resampled,y_resampled=ros.fit_resample(X_train,y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56244, 'high_risk': 56244})

In [24]:
Counter(y_train)

Counter({'low_risk': 56244, 'high_risk': 1908})

In [25]:
# Train the Logistic Regression model using the resampled data
model_os=LogisticRegression(solver='lbfgs',random_state=1)
model_os.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [26]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score,confusion_matrix
y_pred=model_os.predict(X_test)

acc_score=balanced_accuracy_score(y_test,y_pred)
acc_score

0.9954904617260939

In [27]:
# Display the confusion matrix
confusion_matrix(y_test,y_pred)

array([[  590,     2],
       [  106, 18686]], dtype=int64)

In [28]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_pred,y_test))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       1.00      0.85      1.00      0.92      0.92      0.83       696
   low_risk       0.99      1.00      0.85      1.00      0.92      0.86     18688

avg / total       0.99      0.99      0.85      0.99      0.92      0.86     19384



<p>Comment <p> <li> On using Oversampling accuracy marginally improves. 

### SMOTE Oversampling

In [29]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled,y_resampled=SMOTE(random_state=1,sampling_strategy=1.0).fit_resample(X_train,y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56244, 'high_risk': 56244})

In [30]:
# Train the Logistic Regression model using the resampled data
model_smote=LogisticRegression(solver='lbfgs',random_state=1)
model_smote.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [31]:
# Calculated the balanced accuracy score
y_pred=model_smote.predict(X_test)
acc_score_smote=balanced_accuracy_score(y_test,y_pred)
acc_score_smote

0.9954904617260939

In [32]:
# Display the confusion matrix
print(confusion_matrix(y_test,y_pred))

[[  590     2]
 [  106 18686]]


In [33]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      1.00      0.99       592
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18792

avg / total       1.00      0.99      1.00      0.99      1.00      0.99     19384



<p>Comment: <p> <li> precision for high risk is lower than naive random oversampling

# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [34]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc= ClusterCentroids(random_state=1)
X_resampled,y_resampled =cc.fit_resample(X_train,y_train)
# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 1908, 'low_risk': 1908})

In [35]:
# Train the Logistic Regression model using the resampled data
model_cc = LogisticRegression(solver='lbfgs', random_state=1)
model_cc.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [36]:
# Calculate the balanced accuracy score
y_pred = model_cc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9846437673305489

In [37]:
# Display the confusion matrix

confusion_matrix(y_test, y_pred)

array([[  577,    15],
       [  101, 18691]], dtype=int64)

In [38]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      0.97      0.99      0.91      0.98      0.97       592
   low_risk       1.00      0.99      0.97      1.00      0.98      0.97     18792

avg / total       0.99      0.99      0.98      0.99      0.98      0.97     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [39]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
sm=SMOTEENN(random_state=1)
X_resampled,y_resampled=sm.fit_resample(X_train,y_train)
# View the count of target classes with Counter
Counter(y_train)

Counter({'low_risk': 56244, 'high_risk': 1908})

In [40]:
# Train the Logistic Regression model using the resampled data
model_smoteenn=LogisticRegression(solver='lbfgs',random_state=1)
model_smoteenn.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [41]:
# Calculate the balanced accuracy score
y_pred=model_smoteenn.predict(X_test)
acc_score_smoteenn=balanced_accuracy_score(y_test,y_pred)
acc_score_smoteenn

0.995384033458746

In [42]:
# Display the confusion matrix
print(confusion_matrix(y_test,y_pred))

[[  590     2]
 [  110 18682]]


In [43]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      1.00      0.99       592
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18792

avg / total       1.00      0.99      1.00      0.99      1.00      0.99     19384



In [ ]:
# Final Questions

1. Which model had the best balanced accuracy score?

   <p> All the models have very high balanced accuracy score in the range of 0.98 to 0.99. Amongst the models,Random over sampler, smote, smoteenn have 0.995

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.


In [ ]:
print(f'Acc score Smoteenn {acc_score_smoteenn}')
print(f'Acc score Smoteenn {acc_score_smoteenn}')